Download dataset and pre-process
===

In [1]:

import tensorflow_datasets as tfds
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1,2,0"
strategy = tf.distribute.MirroredStrategy()
print("#GPUs",strategy.num_replicas_in_sync)
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 512
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
#BATCH_SIZE=512
print("Batch size",BATCH_SIZE)
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']
#pre-processs
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
#GPUs 1
Batch size 512
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization
from keras.datasets import mnist, cifar10
import numpy as np
from keras.utils import to_categorical
import tensorflow_datasets as tfds
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
#X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
#X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

#one-hot encode target column
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

print("shape: ", X_train.shape)

#convert to TF dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

#preprocess or normalize 
train_dataset=train_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_dataset=validation_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

#for rec in train_dataset.take(5):
    


Using TensorFlow backend.


shape:  (50000, 32, 32, 3)


Build model
==

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, BatchNormalization, Dropout
import numpy as np
from keras.utils import to_categorical
from kerastuner.tuners import BayesianOptimization, Hyperband, RandomSearch
from tensorflow.keras.optimizers import Adam
import tensorflow.keras

def build_model(hp):
    #with strategy.scope():
        model=Sequential()
        model.add(Conv2D(hp.Int("conv_1", min_value=16, max_value=64, step=16), kernel_size=3, activation='relu', input_shape=(32,32,3)))
        model.add(BatchNormalization())
        model.add(Conv2D(128,kernel_size=3,activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(256,kernel_size=3,activation="relu"))
        model.add(Dropout(hp.Choice("rate",values=[0.01,0.02,0.03])))
        model.add(BatchNormalization())
        model.add(Conv2D(512,kernel_size=3,activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(1024,kernel_size=3,activation="relu"))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dense(10,activation='softmax'))
        lr=hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3])
        #optimizers=Adam(learning_rate=lr)
        model.compile(
            optimizer=Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
        
        
EPOCHS = 10
log_dir="/home/salam/cnn/github/logs/hyperband"
SEED=42
EXECUTION_PER_TRIAL=1
stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner = Hyperband(
    build_model,
    max_epochs=EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory=log_dir
   
)

tuner.search_space_summary()


#model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS)

Search space summary
Default search space size: 3
conv_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.02, 0.03], 'ordered': True}
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001], 'ordered': True}


In [5]:
tuner.search(train_dataset, validation_data=validation_dataset, epochs=EPOCHS,  callbacks=[stop_early])

Trial 8 Complete [00h 05m 23s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.1006999984383583
Total elapsed time: 00h 43m 22s

Search: Running Trial #9

Hyperparameter    |Value             |Best Value So Far 
conv_1            |48                |32                
rate              |0.01              |0.01              
learning_rate     |0.1               |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 2/98 [..............................] - ETA: 1:17 - loss: 599.4009 - accuracy: 0.1035WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.5019s vs `on_train_batch_end` time: 1.1207s). Check your callbacks.


98/98 [==============================] - ETA: 0s - loss: 1020.1733 - accuracy: 0.2179WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f83700c4c80> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
98/98 [==============================] - 162s 2s/step - loss: 1020.1733 - accuracy: 0.2179 - val_loss: 29074.7129 - val_accuracy: 0.1000
Epoch 2/2
29/98 [=======>......................] - ETA: 1:41 - loss: 99.5562 - accuracy: 0.2697

KeyboardInterrupt: 

In [5]:
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1: 32
rate: 0.3
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.6335999965667725
Trial summary
Hyperparameters:
conv_1: 64
rate: 0.2
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.6327999830245972
Trial summary
Hyperparameters:
conv_1: 64
rate: 0.2
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.445499986410141
Trial summary
Hyperparameters:
conv_1: 64
rate: 0.3
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.4065000116825104
Trial summary
Hyperparameters:
conv_1: 32
rate: 0.3
learning_rate: 0.1
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.35910001397132874
Trial summary
Hyperparameters:
c


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f34340684a8> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3434068d30>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3434068d30> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f343409aeb8>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f3434068d30> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f343409aeb8>).


ValueError: Shapes (256,) and (10,) are incompatible

In [6]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print( best_hps.get('learning_rate'), best_hps.get('conv_1') ,best_hps.get('rate'))


0.001 32 0.3
